In [1]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import cv2
import math
import wandb

data_wd = Path('/data') / 'raspi_face_detection'
img_path = data_wd / 'images'
labels_path = data_wd / 'labels'

model_sets_path = data_wd / 'model_sets'
aug_sets_path = data_wd / 'aug_data'

# 1 Weights & Biases Configuration

In [2]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project='raspi-facial-recognition',

    # track hyperparameters and run metadata
    config={
        'learning_rate': 0.0001,
        'image_cnt': len(list(img_path.glob('*'))),
        'epochs': 75,
        'batch_size': 128,
        'lr_decay_divider': 0.75,
        'dropout_rate': 0.2,
        'classloss_weight': 0.1,
        'loss_delta_coord_multiplier': 1
        
    }
)

class LRLogger(tf.keras.callbacks.Callback):
    def __init__(self, optimizer):
      super(LRLogger, self).__init__()
      self.optimizer = optimizer

    def on_epoch_end(self, epoch, logs):
      lr = opt._decayed_lr(tf.float32).numpy()
      wandb.log({"lr": lr}, commit=False)

config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: brendanfitz (fitzai). Use `wandb login --relogin` to force relogin


### 2.2 Limit GPU Memory Growth

In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

2024-05-23 11:34:14.550513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-23 11:34:14.555582: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-23 11:34:14.555743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### 5.2 Load Augmented Images to Tensorflow Dataset

In [5]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [6]:
train_images = tf.data.Dataset.list_files(str(aug_sets_path / 'train' / 'images'/ '*.jpg'), shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

2024-05-23 11:34:14.717514: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 11:34:14.718695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-23 11:34:14.718885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-23 11:34:14.719071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [7]:
test_images = tf.data.Dataset.list_files(str(aug_sets_path / 'test' / 'images'/ '*.jpg'), shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [8]:
val_images = tf.data.Dataset.list_files(str(aug_sets_path / 'val' / 'images'/ '*.jpg'), shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [9]:
train_images.as_numpy_iterator().next()

array([[[0.1125    , 0.15122549, 0.34191176],
        [0.0908701 , 0.1810049 , 0.40667892],
        [0.09791667, 0.16599265, 0.39558825],
        ...,
        [0.99607843, 1.        , 1.        ],
        [0.997549  , 0.9906863 , 1.        ],
        [0.9352941 , 0.90392154, 0.95686275]],

       [[0.11709559, 0.17003676, 0.34797794],
        [0.0911152 , 0.18768382, 0.41121325],
        [0.08731618, 0.16182598, 0.38927695],
        ...,
        [0.99607843, 1.        , 1.        ],
        [0.9965686 , 0.9901348 , 1.        ],
        [0.9355392 , 0.90416664, 0.95710784]],

       [[0.07567402, 0.1286152 , 0.30655637],
        [0.08474265, 0.17830883, 0.41084558],
        [0.09460784, 0.16911764, 0.40042892],
        ...,
        [0.99264705, 1.        , 0.9965686 ],
        [0.9995098 , 0.99264705, 1.        ],
        [0.9356005 , 0.9042279 , 0.9571691 ]],

       ...,

       [[0.08511029, 0.11164216, 0.15116422],
        [0.09019608, 0.11764706, 0.15686275],
        [0.09013481, 0

# 6. Prepare Labels

### 6.1 Build Label Loading Function

In [10]:
import itertools

In [11]:
# fn = '49e246c6-0968-11ef-b027-dca632a68397_5.json' # both
# fn = '3a38b642-0968-11ef-b027-dca632a68397_4.json' # kara
# fn = '4a2f0b44-0965-11ef-a3f3-dca632a68397_37.json' # brendan
fn = '369aab30-0968-11ef-b027-dca632a68397_0.json' # error
label_path = aug_sets_path / 'test' / 'labels' / fn
with open(label_path, 'r', encoding = "utf-8") as f:
    label = json.load(f)

label['classes'], label['bboxes']

([2],
 [[0, 0, 0, 0],
  [0.053381995133820005,
   0.316139497161395,
   0.20531494998648275,
   0.5172749391727494]])

In [12]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)

    return [int(x in label['classes']) for x in range(1, 3)], list(itertools.chain(*label['bboxes']))

# classes, bboxes = load_labels(label_path)

### 6.2 Load Labels to Tensorflow Dataset

In [13]:
train_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'train' / 'labels'/ '*.json'), shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [14]:
test_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'test' / 'labels'/ '*.json'), shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [15]:
val_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'val' / 'labels'/ '*.json'), shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

# 7. Combine Label and Image Samples

### 7.1 Check Partition Lengths

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

### 7.2 Create Final Datasets (Images/Labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(len(train_images))
train = train.batch(config.batch_size)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(len(test_images))
test = test.batch(config.batch_size)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(len(val_images))
val = val.batch(config.batch_size)
val = val.prefetch(4)

### 7.3 View Images and Annotations

In [ ]:
data_samples = train.as_numpy_iterator()

In [ ]:
res = data_samples.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    sample_labels = ', '.join(res[1][0][idx].astype(str).tolist())

    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)

    ax[idx].imshow(sample_image)
    ax[idx].set_title(sample_labels)

# 8. Build Deep Learning using the Functional API

### 8.1 Import Layers and Base Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D, Dropout
from tensorflow.keras.applications import VGG16

### 8.2 Download VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
vgg.summary()

### 8.3 Build instance of Network

In [ ]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class1 = tf.keras.layers.Dropout(config.dropout_rate)(class1)
    class2 = Dense(2, activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress1 = Dropout(config.dropout_rate)(regress1)
    regress2 = Dense(8, activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

### 8.4 Test out Neural Network

In [ ]:
facetracker = build_model()

In [ ]:
facetracker.summary()

In [ ]:
X, y = train.as_numpy_iterator().next()

# 9. Define Losses and Optimizers

### 9.1 Define Optimizer and LR

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./config.lr_decay_divider -1)/batches_per_epoch
lr_decay

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, decay=lr_decay)

### 9.2 Create Localization Loss and Classification Loss

In [ ]:
def localization_loss(y_classes, y_coords, yhat_coords):
    yhat_coords = tf.concat(
        (yhat_coords[:, :4] * tf.cast(y_classes[:, 0:1], tf.float32),
         yhat_coords[:, 4:] * tf.cast(y_classes[:, 1:2], tf.float32)),
        axis=1
    )
    # differences between coordinates
    delta_coord = tf.reduce_sum(tf.square(y_coords[:,:2] - yhat_coords[:,:2])) + tf.reduce_sum(tf.square(y_coords[:,4:6] - yhat_coords[:,4:6]))

    # differences between size
    delta_size = 0
    for i in range(2):
        h_true = y_coords[:,3+4*i] - y_coords[:,1+4*i] 
        w_true = y_coords[:,2+4*i] - y_coords[:,0+4*i] 
    
        h_pred = yhat_coords[:,3+4*i] - yhat_coords[:,1+4*i] 
        w_pred = yhat_coords[:,2+4*i] - yhat_coords[:,0+4*i] 
        
        delta_size += tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord * config.loss_delta_coord_multiplier + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss

# 10. Train Neural Network

### 10.1 Create Custom Model Class

In [ ]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def lossfn(self, y_classes, y_coords, yhat_classes, yhat_coords):
        batch_classloss = self.closs(y_classes, yhat_classes)
        batch_localizationloss = self.lloss(tf.cast(y_classes, tf.float32), tf.cast(y_coords, tf.float32), yhat_coords)
        
        total_loss = batch_localizationloss + config.classloss_weight*batch_classloss
        return total_loss, batch_classloss, batch_localizationloss
    
    def train_step(self, batch, **kwargs): 
        
        X, (y_classes, y_coords) = batch
        
        with tf.GradientTape() as tape: 
            yhat_classes, yhat_coords = self.model(X, training=True)
            
            total_loss, batch_classloss, batch_localizationloss = self.lossfn(y_classes, y_coords, yhat_classes, yhat_coords)
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, (y_classes, y_coords) = batch
        
        yhat_classes, yhat_coords = self.model(X, training=False)
        total_loss, batch_classloss, batch_localizationloss = self.lossfn(y_classes, y_coords, yhat_classes, yhat_coords)
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

model = FaceTracker(facetracker)

model.compile(opt, classloss, regressloss)

### 10.2 Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=config.epochs, validation_data=val, 
                 callbacks=[tensorboard_callback, wandb.keras.WandbMetricsLogger(), LRLogger(opt)])

run.finish()

### 10.3 Plot Performance

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

# 11. Make Predictions

### 11.1 Make Predictions on Test Set

In [ ]:
test_data = val.as_numpy_iterator()
test_sample = test_data.next()

In [ ]:
test_sample = test_data.next()

yhat = facetracker.predict(test_sample[0])

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx][0] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    if yhat[0][idx][1] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)
    
    ax[idx].imshow(sample_image)
    ax[idx].set_title(', '.join([f'{x:,.0%}' for x in yhat[0][idx]]))

### 11.2 Save the Model

In [ ]:
facetracker.save('facetracker.h5')